In [1]:
import numpy as np
import pandas as pd
from operator import itemgetter

In [2]:
y_at = pd.DataFrame({"Outlook" :["Sunny","Sunny","Overcast","Rain","Rain","Rain","Overcast","Sunny","Sunny","Rain","Sunny","Overcast","Overcast","Rain"],"Temperature":["Hot","Hot","Hot","Mild","Cool","Cool","Cool","Mild","Cool","Mild","Mild","Mild","Hot","Mild"],"Humidity":["High","High","High","High","Normal", "Normal","Normal","High","Normal","Normal","Normal","High","Normal","High"],"Wind": ["Weak", "Strong", "Weak", "Weak", "Weak", "Strong","Strong", "Weak", "Weak", "Weak", "Strong","Strong", "Weak","Strong"]})

In [3]:
y = pd.DataFrame(["No", "No", "yes", "yes", "yes","No", "yes","No", "yes", "yes", "yes", "yes", "yes","No"])

In [ ]:
def entropy(y):
    out = np.array(y)
    ent = 0
    for val in np.unique(out):
        temp = -(len(out[out == val])/len(y))*(np.log2(len(out[out == val])/len(y)))
        ent += temp
    return ent
        

In [ ]:
def get_IG(y,att_y):
    att_y.columns = range(att_y.shape[1])
    temp_ig = 0
    
    for att in att_y[0].unique():
        
        temp_df = att_y[att_y[0] == att]
        H_x = entropy(temp_df[1])
        p_s = len(temp_df)/len(att_y)
        temp_ig += ((p_s*H_x))
        
    return entropy(y)-temp_ig

In [ ]:
#test entropy calculation
entropy(y)

In [ ]:
def get_top_node(data,y):
    temp_df = pd.concat([data,y],axis = 1)
    temp_IGs = []
    for att in data.keys():
        single_df = temp_df[[att,y.keys()[0]]]
        IG = get_IG(y,single_df)
        temp_IGs.append((att, IG))
    
    return sorted(temp_IGs, key = itemgetter(1), reverse = True)[0]
    

In [ ]:
get_top_node(y_at,pd.DataFrame(y))

In [ ]:
def set_branches(root,sub_tree,data, y):
    sub_tree[root] = {}
    for attr in full_data[root].unique():
            print("Working for attribute:",attr,"for node:",root)
            get_next_node(root,attr,data,y)
            


In [ ]:
def get_next_node(current_root, attr, data, y):
    full_data = pd.concat([data,y],axis = 1)
    temp_data = data[data[current_root] == attr]
    temp_data_full = full_data[full_data[current_root] == attr]
    if len(temp_data_full[temp_data_full.columns[-1]].unique()) == 1:
        node = (temp_data_full[temp_data_full.columns[-1]].unique(),1)
        print("Node found", node)
        return node
    else:
        current_node = get_top_node(temp_data,temp_data_full[temp_data_full.columns[-1]])[0]
        print("Next node for working:",current_node)
        return (current_node,0)

In [ ]:
def fit_tree(data,target):
    full_data = pd.concat([data,target],axis = 1)
    tree = {}
    #get root node
    n = data.shape[0] - 1
    root = get_top_node(data,target)[0]
    print("Root Selected:", root)
    #get attributes of current node
    
    for column in data.keys():
        
        final_tree = set_branches(root,tree,data,full_data)
   
    
    return final_tree
    #check if a sub branch gives a defined answer
    
    #redo calculations for other subbranches
    

In [ ]:
tree = fit_tree(y_at,y)

In [ ]:
tree

In [ ]:
get_IG(y,y_at)

# Decision Tree from Sklearn

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OrdinalEncoder

In [5]:
dt = DecisionTreeClassifier()

In [6]:
ord_enc = OrdinalEncoder()
data = ord_enc.fit_transform(y_at[["Outlook","Temperature", "Humidity", "Wind"]])
data

array([[2., 1., 0., 1.],
       [2., 1., 0., 0.],
       [0., 1., 0., 1.],
       [1., 2., 0., 1.],
       [1., 0., 1., 1.],
       [1., 0., 1., 0.],
       [0., 0., 1., 0.],
       [2., 2., 0., 1.],
       [2., 0., 1., 1.],
       [1., 2., 1., 1.],
       [2., 2., 1., 0.],
       [0., 2., 0., 0.],
       [0., 1., 1., 1.],
       [1., 2., 0., 0.]])

In [7]:
out = ord_enc.fit_transform(y)

In [8]:
dt.fit(data,out)

DecisionTreeClassifier()

In [12]:
from sklearn.tree import export_graphviz
from six import StringIO  
from IPython.display import Image, display
import pydotplus

dot_data = StringIO()
export_graphviz(dt, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,feature_names = ["Outlook","Temperature", "Humidity", "Wind"],class_names=['0','1'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  

display(Image(graph.create_png()))

InvocationException: GraphViz's executables not found

In [14]:
dt.predict(np.array([1,2,1,2]))

ValueError: Expected 2D array, got 1D array instead:
array=[1. 2. 1. 2.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.